In [106]:
from sklearn.datasets import fetch_mldata
from sklearn.neural_network import MLPClassifier
from sklearn import preprocessing
from sklearn.feature_extraction import DictVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.externals import joblib
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from numpy import array
from numpy import argmax

In [107]:
import pandas as pd 
from pandas import DataFrame, read_csv
import numpy as np
import matplotlib.pyplot as plt
# Enable inline plotting
%matplotlib inline
np.set_printoptions(suppress=True)

DISPLAY_MAX_ROWS = 20  # number of max rows to print for a DataFrame
pd.set_option('display.max_rows', DISPLAY_MAX_ROWS)

In [108]:
train = read_csv('data/train.csv')
test = read_csv('data/test.csv')

In [109]:
X_train = train[train.columns.difference(['PassengerId', 'Survived', 'Name', 'Cabin', 'Age', 'Embarked', 'Ticket'])]  # independent variables data
y_train = train.Survived  # dependednt variable data
X_test = test[test.columns.difference(['PassengerId', 'Survived', 'Name', 'Cabin', 'Age', 'Embarked', 'Ticket'])]  # independent variables data

In [110]:
def one_hot_dataframe(data, cols, replace=False):
    vec = DictVectorizer()
    mkdict = lambda row: dict((col, row[col]) for col in cols)
    vecData = pd.DataFrame(vec.fit_transform(data[cols].apply(mkdict, axis=1)).toarray())
    vecData.columns = vec.get_feature_names()
    vecData.index = data.index
    if replace is True:
        data = data.drop(cols, axis=1)
        data = data.join(vecData)
    return (data, vecData, vec)

In [111]:
def one_hot_encode(data): 
    # data example
#     data = ['cold', 'cold', 'warm', 'cold', 'hot', 'hot', 'warm', 'cold', 'warm', 'hot']
    values = array(data)

    # integer encode
    label_encoder = LabelEncoder()
    integer_encoded = label_encoder.fit_transform(values)
    
    # binary encode
    onehot_encoder = OneHotEncoder(sparse=False)
    integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
    onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
    return onehot_encoded

In [112]:
X_train2, _, _ = one_hot_dataframe(X_train, ['Sex'], replace=True)
X_train2 = X_train2.where((pd.notnull(X_train2)), 100)

X_test2, _, _ = one_hot_dataframe(X_test, ['Sex', 'Pclass'], replace=True)
X_test2 = X_test2.where((pd.notnull(X_test2)), 100)

In [113]:
values = array(X_test2.Pclass)
# integer encode
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(values)
print(pclass[11,:])
print(integer_encoded)
print(X_test2.Pclass)

[1. 0. 0.]
[2 2 1 2 2 2 2 1 2 2 2 0 0 1 0 1 1 2 2 2 0 2 0 0 0 2 0 2 0 2 1 1 2 2 0 2 2
 2 2 2 2 0 2 1 0 2 0 2 0 2 0 1 1 0 1 2 2 2 2 0 2 1 2 2 0 1 2 0 0 0 2 2 2 0
 0 0 2 0 1 2 2 0 0 2 1 2 2 2 2 1 2 2 0 2 0 2 0 2 2 2 0 1 2 2 2 2 2 2 2 1 1
 2 0 2 0 2 2 2 0 1 1 2 0 2 2 2 2 2 1 2 2 0 2 2 2 2 2 1 2 2 2 0 0 1 0 2 0 2
 0 1 0 2 2 2 2 2 0 2 0 2 2 2 1 2 1 2 0 2 0 2 2 2 2 2 2 1 1 0 1 0 1 0 0 2 0
 1 1 2 2 1 1 0 2 1 1 2 0 2 1 2 2 2 0 1 1 0 2 1 0 2 2 2 1 1 2 0 2 0 0 2 1 2
 1 2 0 2 2 2 2 1 1 0 2 2 0 2 0 2 1 0 0 1 0 2 2 0 1 1 1 2 1 2 0 2 2 2 2 2 1
 2 2 2 1 2 1 2 0 2 2 2 0 2 0 2 2 1 1 1 1 1 2 2 2 2 2 2 2 0 2 2 0 2 2 0 2 2
 1 2 0 2 2 1 1 2 2 0 0 2 0 2 2 2 2 2 0 2 0 1 2 1 2 2 1 0 0 2 1 0 1 1 1 0 2
 2 2 0 1 2 1 2 1 2 2 0 2 2 1 2 1 1 0 1 1 1 2 0 0 2 2 2 2 1 1 2 0 2 2 2 0 1
 1 0 0 1 0 0 2 1 0 2 2 2 2 2 1 1 2 1 2 2 0 0 2 1 2 0 2 0 2 2 0 1 0 0 0 1 1
 0 2 2 2 0 2 2 0 2 2 2]
0      3.0
1      3.0
2      2.0
3      3.0
4      3.0
5      3.0
6      3.0
7      2.0
8      3.0
9      3.0
      ... 
408    3.0
409    3

In [114]:
pclass = one_hot_encode(X_train2.Pclass)
# X_test2.drop('Pclass', axis=1, inplace=True)
X_train2['Pclass=1'] = pclass[:, 0]
X_train2['Pclass=2'] = pclass[:, 1]
X_train2['Pclass=3'] = pclass[:, 2]
X_train2.drop('Pclass', axis=1, inplace=True)

In [115]:
pclass = one_hot_encode(X_test2.Pclass)
# X_test2.drop('Pclass', axis=1, inplace=True)
X_test2['Pclass=1'] = pclass[:, 0]
X_test2['Pclass=2'] = pclass[:, 1]
X_test2['Pclass=3'] = pclass[:, 2]
X_test2.drop('Pclass', axis=1, inplace=True)

In [116]:
X_test2

,Fare,Parch,SibSp,Sex=female,Sex=male,Pclass=1,Pclass=2,Pclass=3
0,7.8292,0,0,0.0,1.0,0.0,0.0,1.0
1,7.0000,0,1,1.0,0.0,0.0,0.0,1.0
2,9.6875,0,0,0.0,1.0,0.0,1.0,0.0
3,8.6625,0,0,0.0,1.0,0.0,0.0,1.0
4,12.2875,1,1,1.0,0.0,0.0,0.0,1.0
5,9.2250,0,0,0.0,1.0,0.0,0.0,1.0
6,7.6292,0,0,1.0,0.0,0.0,0.0,1.0
7,29.0000,1,1,0.0,1.0,0.0,1.0,0.0
8,7.2292,0,0,1.0,0.0,0.0,0.0,1.0
9,24.1500,0,2,0.0,1.0,0.0,0.0,1.0


In [117]:
# X_train2.name.unique()
X_train2[['Fare']] = StandardScaler().fit_transform(X_train2[['Fare']])
X_test2[['Fare']] = StandardScaler().fit_transform(X_test2[['Fare']])

In [118]:
X_train2

,Fare,Parch,SibSp,Sex=female,Sex=male,Pclass=1,Pclass=2,Pclass=3
0,-0.502445,0,1,0.0,1.0,0.0,0.0,1.0
1,0.786845,0,1,1.0,0.0,1.0,0.0,0.0
2,-0.488854,0,0,1.0,0.0,0.0,0.0,1.0
3,0.420730,0,1,1.0,0.0,1.0,0.0,0.0
4,-0.486337,0,0,0.0,1.0,0.0,0.0,1.0
5,-0.478116,0,0,0.0,1.0,0.0,0.0,1.0
6,0.395814,0,0,0.0,1.0,1.0,0.0,0.0
7,-0.224083,1,3,0.0,1.0,0.0,0.0,1.0
8,-0.424256,2,0,1.0,0.0,0.0,0.0,1.0
9,-0.042956,0,1,1.0,0.0,0.0,1.0,0.0


In [119]:
# plt.hist(X_test2[['Pclass']], bins=np.arange(data.min(), data.max()+1))
# X_test2.groupby('Pclass').count()

In [120]:
enc = preprocessing.OneHotEncoder(n_values=[2, 3, 4])
# Note that there are missing categorical values for the 2nd and 3rd
# features
enc.fit([[1, 2, 3], [0, 2, 0]])  
OneHotEncoder(categorical_features='all',
       handle_unknown='error', n_values=[2, 3, 4], sparse=True)
enc.transform([[1, 0, 0]]).toarray()

array([[0., 1., 1., 0., 0., 1., 0., 0., 0.]])

In [121]:
# define example
data = ['cold', 'cold', 'warm', 'cold', 'hot', 'hot', 'warm', 'cold', 'warm', 'hot']
values = array(data)
print(values)
# integer encode
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(values)
print(integer_encoded)
# binary encode
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
print(onehot_encoded)
# invert first example
inverted = label_encoder.inverse_transform([argmax(onehot_encoded[0, :])])
print(inverted)

['cold' 'cold' 'warm' 'cold' 'hot' 'hot' 'warm' 'cold' 'warm' 'hot']
[0 0 2 0 1 1 2 0 2 1]
[[1. 0. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 1. 0.]]
['cold']


/home/aung/.conda/envs/mytensorflow/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [122]:
# mnist = fetch_mldata("MNIST original")

# # rescale the data, use the traditional train/test split
# X, y = mnist.data / 255., mnist.target
# X_train, X_test = X[:60000], X[60000:]
# y_train, y_test = y[:60000], y[60000:]

In [123]:
# mlp = MLPClassifier(hidden_layer_sizes=(100, 100), max_iter=400, alpha=1e-4,
#                     solver='sgd', verbose=10, tol=1e-4, random_state=1)
mlp = MLPClassifier(hidden_layer_sizes=(100, 100), 
                    max_iter=1000, 
                    alpha=1e-4,
                    solver='sgd', 
                    verbose=10, 
                    tol=1e-4, 
                    random_state=1,
                    learning_rate_init=.001, 
                    validation_fraction=0.2)

mlp.fit(X_train2, y_train)

# pipe = make_pipeline(StandardScaler(), mlp)
# pipe.fit(X_train2, y_train)

joblib.dump(mlp, 'titanic_model_mlp.pkl')
print("Training set score: %f" % mlp.score(X_train2, y_train))
# print("Test set score: %f" % mlp.score(X_test, y_test))


Iteration 1, loss = 0.66518141
Iteration 2, loss = 0.66254437
Iteration 3, loss = 0.65868383
Iteration 4, loss = 0.65397921
Iteration 5, loss = 0.64912992
Iteration 6, loss = 0.64427664
Iteration 7, loss = 0.63943901
Iteration 8, loss = 0.63460859
Iteration 9, loss = 0.63016245
Iteration 10, loss = 0.62563275
Iteration 11, loss = 0.62128710
Iteration 12, loss = 0.61703028
Iteration 13, loss = 0.61296390
Iteration 14, loss = 0.60902365
Iteration 15, loss = 0.60531322
Iteration 16, loss = 0.60159057
Iteration 17, loss = 0.59803235
Iteration 18, loss = 0.59471319
Iteration 19, loss = 0.59143357
Iteration 20, loss = 0.58834003
Iteration 21, loss = 0.58536916
Iteration 22, loss = 0.58232281
Iteration 23, loss = 0.57948311
Iteration 24, loss = 0.57657192
Iteration 25, loss = 0.57377389
Iteration 26, loss = 0.57104421
Iteration 27, loss = 0.56843407
Iteration 28, loss = 0.56583557
Iteration 29, loss = 0.56333673
Iteration 30, loss = 0.56079240
Iteration 31, loss = 0.55839827
Iteration 32, los

In [124]:
mlp = joblib.load('titanic_model_mlp.pkl')
p = mlp.predict(X_test2)
predictions = pd.DataFrame(data={'PassengerId': test.PassengerId, 'Survived': p})

In [125]:
predictions

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1
5,897,0
6,898,1
7,899,0
8,900,1
9,901,0


In [126]:
predictions.to_csv('predictions/result.csv', sep=',', encoding='utf-8', index=False)